In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
prefix = os.getcwd().split("/src")[0]
src_dir = os.path.join(prefix, "src")
data_dir = os.path.join(prefix, "dataset")
sys.path.append(src_dir)

In [2]:
from torch import nn
from models import VisuelleDataset
from utils import misc_utils
import torch
import mlflow
from torch import optim
from tqdm import tqdm

/Users/tccuong1404/Documents/Projects/visuelle-sales-forecasting/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/tccuong1404/Documents/Projects/visuelle-sales-forecasting/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
vis_dataset = VisuelleDataset(data_dir)

2024-02-16 14:01:42.986 | INFO     | models.visuelle_dataset:__init__:32 - ====== DATASET ======
2024-02-16 14:01:42.986 | INFO     | models.visuelle_dataset:__init__:33 - Train path: /Users/tccuong1404/Documents/Projects/visuelle-sales-forecasting/dataset/train.csv
2024-02-16 14:01:42.987 | INFO     | models.visuelle_dataset:__init__:34 - Test path: /Users/tccuong1404/Documents/Projects/visuelle-sales-forecasting/dataset/test.csv
2024-02-16 14:01:42.987 | INFO     | models.visuelle_dataset:__init__:35 - Category path: /Users/tccuong1404/Documents/Projects/visuelle-sales-forecasting/dataset/category_labels.pt
2024-02-16 14:01:42.987 | INFO     | models.visuelle_dataset:__init__:36 - Color path: /Users/tccuong1404/Documents/Projects/visuelle-sales-forecasting/dataset/color_labels.pt
2024-02-16 14:01:42.987 | INFO     | models.visuelle_dataset:__init__:37 - Fabric path: /Users/tccuong1404/Documents/Projects/visuelle-sales-forecasting/dataset/fabric_labels.pt


In [4]:
train_loader = vis_dataset.get_data_loader(train=True)
test_loader = vis_dataset.get_data_loader(train=False)

100%|##########| 497/497 [00:00<00:00, 838.72it/s]


#### Test the Input and Output

In [5]:
targets, attrs, temporals, gtrends, vtrends = next(iter(train_loader))

In [6]:
batch_size = 32
cat = vtrends.view(batch_size, -1, 3)
col = vtrends[:,1,:].view(batch_size, -1, 1)
fab = vtrends[:,2,:].view(batch_size, -1, 1)

In [7]:
encoder = nn.LSTM(3, 12, batch_first=True)

In [8]:
targets, states = encoder(vtrends.view(batch_size, -1, 3))

#### Try LSTM with VTrend and GTrend output directly

In [22]:
class LSTMForecast(nn.Module):
    def __init__(self, output_dim: int, n_trend: int):
        super(LSTMForecast, self).__init__()
        self.vtrend_encoder = nn.LSTM(n_trend, output_dim, batch_first=True)
        self.gtrend_encoder = nn.LSTM(n_trend, output_dim, batch_first=True)
        self.n_trend = n_trend
        self.relu = nn.ReLU()
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=0.1)

        
    def forward(self, vtrends, gtrends, targets):
        batch_size = targets.size(0)
        vtrend_out, _ = self.vtrend_encoder(vtrends.view(batch_size, -1, self.n_trend))
        gtrend_out, _ = self.gtrend_encoder(gtrends.view(batch_size, -1, self.n_trend))
        out = torch.stack([vtrend_out[:,-1], gtrend_out[:,-1]]).mean(dim=0)
        loss = self.criterion(targets, out)
        return loss
    

    def predict(self, vtrends, gtrends):
        batch_size = vtrends.size(0)
        vtrend_out, _ = self.vtrend_encoder(vtrends.view(batch_size, -1, self.n_trend))
        gtrend_out, _ = self.gtrend_encoder(gtrends.view(batch_size, -1, self.n_trend))
        out = torch.stack([vtrend_out[:,-1], gtrend_out[:,-1]]).mean(dim=0)
        out = self.relu(out)
        return out
    

    def train(self, data_loader, test_loader, wape_call, epoch: int):
        batch_bar = tqdm(data_loader, desc=f"Epoch {0}, loss={0:.5f}")
        for i in range(epoch):
            batch_bar.reset()
            sum_loss = 0
            for batch in data_loader:
                targets, _, _, vtrends, gtrends = batch
                self.optimizer.zero_grad()
                loss = self.forward(
                    vtrends, gtrends, targets
                )
                loss.backward()
                sum_loss += loss
                self.optimizer.step()
                batch_bar.update()
                batch_bar.refresh()

            batch_bar.set_description(f"Epoch {i}, loss={sum_loss:.5f}")

            targets, _, _, vtrends, gtrends = test_loader.dataset[:]
            outputs = self.predict(vtrends, gtrends)
            mlflow.log_metric("val_wape", wape_call(targets, outputs), step=i)

            targets, _, _, vtrends, gtrends = data_loader.dataset[:]
            outputs = self.predict(vtrends, gtrends)
            mlflow.log_metric("train_wape", wape_call(targets, outputs), step=i)

In [18]:

mlflow.start_run()

<ActiveRun: >

In [23]:
model = LSTMForecast(output_dim=12, n_trend=4)

In [24]:
model.parameters()

<generator object Module.parameters at 0x17778bc10>

In [25]:
model.train(train_loader, test_loader, misc_utils.cal_wape, 50)

Epoch 49, loss=0.32158: 100%|██████████| 128/128 [00:03<00:00, 38.12it/s]


In [26]:
mlflow.end_run()

#### Encoder-Decoder architecture

In [10]:
class LSTMForecastED(nn.Module):
    def __init__(self, hidden_dim: int, output_dim: int):
        super(LSTMForecastED, self).__init__()
        self.n_out = 12
        self.vtrend_encoder = nn.LSTM(4, hidden_dim, batch_first=True)
        self.gtrend_encoder = nn.LSTM(4, hidden_dim, batch_first=True)
        self.decoder = nn.LSTM(1, hidden_dim, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=0.1)

    def forward(self, vtrends, gtrends, targets):
        batch_size = targets.size(0) # Batch size
        target_len = targets.size(1) # Sequence length
        _, state = self.gtrend_encoder(gtrends.view(batch_size, -1, 4))
        _, state = self.vtrend_encoder(vtrends.view(batch_size, -1, 4), state)
        loss = 0
        dec_in = torch.zeros((batch_size, 1))
        for i in range(target_len):
            dec_out, state = self.decoder(dec_in.view(batch_size, 1, -1), state)
            dec_in = self.fc_out(dec_out.view(batch_size, -1))
            ground_truth = targets[:,i].view(batch_size, -1)
            loss += self.criterion(dec_in, ground_truth)
        return loss
    
    def predict(self, vtrends, gtrends):
        batch_size = vtrends.size(0) # Batch size
        output = []
        _, state = self.gtrend_encoder(gtrends.view(batch_size, -1, 4))
        _, state = self.vtrend_encoder(vtrends.view(batch_size, -1, 4), state)
        dec_in = torch.zeros((batch_size, 1))
        for _ in range(self.n_out):
            dec_out, state = self.decoder(dec_in.view(batch_size, 1, -1), state)
            dec_in = self.fc_out(dec_out.view(batch_size, -1))
            output.append(dec_in)
        return torch.hstack(output)
    
    
    def train(self, data_loader, test_loader, wape_call, epoch: int):
        batch_bar = tqdm(data_loader, desc=f"Epoch {0}, loss={0:.5f}")
        for i in range(epoch):
            batch_bar.reset()
            sum_loss = 0
            for batch in data_loader:
                targets, _, _, gtrends, vtrends = batch
                self.optimizer.zero_grad()
                loss = self.forward(
                    vtrends, gtrends, targets
                )
                loss.backward()
                sum_loss += loss
                self.optimizer.step()
                batch_bar.update()
                batch_bar.refresh()

            batch_bar.set_description(f"Epoch {i}, loss={sum_loss:.5f}")

            targets, _, _, gtrends, vtrends = test_loader.dataset[:]
            outputs = self.predict(vtrends, gtrends)
            mlflow.log_metric("val_wape", wape_call(targets, outputs), step=i)

            targets, _, _, gtrends, vtrends = data_loader.dataset[:]
            outputs = self.predict(vtrends, gtrends)
            mlflow.log_metric("train_wape", wape_call(targets, outputs), step=i)
        

In [7]:
import mlflow
mlflow.start_run()

<ActiveRun: >

In [11]:
model = LSTMForecastED(32, 1)

In [12]:
model.train(train_loader, test_loader, misc_utils.cal_wape, 50)

Epoch 49, loss=3.84129: 100%|██████████| 128/128 [00:04<00:00, 30.94it/s]


In [12]:
mlflow.end_run()

#### VTrend Only

In [6]:
class LSTMForecastVO(nn.Module):
    def __init__(self, hidden_dim: int, output_dim: int):
        super(LSTMForecastVO, self).__init__()
        self.n_out = 12
        self.encoder = nn.LSTM(4, hidden_dim, batch_first=True)
        self.decoder = nn.LSTM(1, hidden_dim, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=0.1)

    def forward(self, vtrends, targets):
        batch_size = targets.size(0) # Batch size
        target_len = targets.size(1) # Sequence length
        _, state = self.encoder(vtrends.view(batch_size, -1, 4))
        loss = 0
        dec_in = torch.zeros((batch_size, 1))
        for i in range(target_len):
            dec_out, state = self.decoder(dec_in.view(batch_size, 1, -1), state)
            dec_in = self.fc_out(dec_out.view(batch_size, -1))
            ground_truth = targets[:,i].view(batch_size, -1)
            loss += self.criterion(dec_in, ground_truth)
        return loss
    
    def predict(self, vtrends):
        batch_size = vtrends.size(0) # Batch size
        output = []
        _, state = self.encoder(vtrends.view(batch_size, -1, 4))
        dec_in = torch.zeros((batch_size, 1))
        for _ in range(self.n_out):
            dec_out, state = self.decoder(dec_in.view(batch_size, 1, -1), state)
            dec_in = self.fc_out(dec_out.view(batch_size, -1))
            output.append(dec_in)
        return torch.hstack(output)
    
    
    def train(self, data_loader, test_loader, wape_call, epoch: int):
        batch_bar = tqdm(data_loader, desc=f"Epoch {0}, loss={0:.5f}")
        for i in range(epoch):
            batch_bar.reset()
            sum_loss = 0
            for batch in data_loader:
                targets, _, _, gtrends, vtrends = batch
                self.optimizer.zero_grad()
                loss = self.forward(
                    vtrends, targets
                )
                loss.backward()
                sum_loss += loss
                self.optimizer.step()
                batch_bar.update()
                batch_bar.refresh()

            batch_bar.set_description(f"Epoch {i}, loss={sum_loss:.5f}")

            targets, _, _, gtrends, vtrends = test_loader.dataset[:]
            outputs = self.predict(vtrends)
            mlflow.log_metric("val_wape", wape_call(targets, outputs), step=i)

            targets, _, _, gtrends, vtrends = data_loader.dataset[:]
            outputs = self.predict(vtrends)
            mlflow.log_metric("train_wape", wape_call(targets, outputs), step=i)
        

In [7]:
mlflow.start_run()
model = LSTMForecastVO(32, 1)

In [8]:
model.train(train_loader, test_loader, misc_utils.cal_wape, 50)

Epoch 49, loss=3.87457: 100%|██████████| 128/128 [00:03<00:00, 39.00it/s]


In [9]:
mlflow.end_run()

#### LSTM No Encoder Only

In [21]:
class LSTMForecastVEO(nn.Module):
    def __init__(self, output_dim: int, n_trend: int):
        super(LSTMForecastVEO, self).__init__()
        self.encoder = nn.LSTM(n_trend, output_dim, batch_first=True)
        self.n_trend = n_trend
        self.relu = nn.ReLU()
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=0.1)

        
    def forward(self, vtrends, gtrends, targets):
        batch_size = targets.size(0)
        out, _ = self.encoder(vtrends.view(batch_size, -1, self.n_trend))
        loss = self.criterion(targets, out[:,-1])
        return loss
    

    def predict(self, vtrends, gtrends):
        batch_size = vtrends.size(0)
        out, _ = self.encoder(vtrends.view(batch_size, -1, self.n_trend))
        out = self.relu(out[:,-1])
        return out
    

    def train(self, data_loader, test_loader, wape_call, epoch: int):
        batch_bar = tqdm(data_loader, desc=f"Epoch {0}, loss={0:.5f}")
        for i in range(epoch):
            batch_bar.reset()
            sum_loss = 0
            for batch in data_loader:
                targets, _, _, vtrends, gtrends = batch
                self.optimizer.zero_grad()
                loss = self.forward(
                    vtrends, gtrends, targets
                )
                loss.backward()
                sum_loss += loss
                self.optimizer.step()
                batch_bar.update()
                batch_bar.refresh()

            batch_bar.set_description(f"Epoch {i}, loss={sum_loss:.5f}")

            targets, _, _, vtrends, gtrends = test_loader.dataset[:]
            outputs = self.predict(vtrends, gtrends)
            mlflow.log_metric("val_wape", wape_call(targets, outputs), step=i)

            targets, _, _, vtrends, gtrends = data_loader.dataset[:]
            outputs = self.predict(vtrends, gtrends)
            mlflow.log_metric("train_wape", wape_call(targets, outputs), step=i)

In [26]:
mlflow.start_run()

<ActiveRun: >

In [27]:
model = LSTMForecastVEO(12, 4)

In [28]:
model.train(train_loader, test_loader, misc_utils.cal_wape, 50)

In [29]:
mlflow.end_run()